In [1]:
import sys
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

sys.path.append(os.path.abspath("../")) 

from utils.data_utils import DeepFashionSubsetDataset
from models.models import MyCustomCNN
from torchvision import transforms
from torch.utils.data import DataLoader 

In [3]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225]),
])



In [5]:
train_csv = "../data/subset/train/classification_metadata.csv"
train_img_folder = "../data/subset/train/images"

val_csv = "../data/subset/val/classification_metadata.csv"
val_img_folder = "../data/subset/val/images"

train_dataset = DeepFashionSubsetDataset(
    csv_file=train_csv,
    images_folder=train_img_folder,
    transform=train_transform,
    use_bbox=True
)

val_dataset = DeepFashionSubsetDataset(
    csv_file=val_csv,
    images_folder=val_img_folder,
    transform=val_transform,
    use_bbox=True
)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
print(f"Number of training samples:   {len(train_dataset)}")
print(f"Number of validation samples: {len(val_dataset)}")

Number of training samples:   6461
Number of validation samples: 1300


## Define Model

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
model = MyCustomCNN(num_classes=13)
model = model.to(device)

Using device: cpu


In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [8]:
num_epochs = 5  

# Training
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:

        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backprop and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate loss
        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)

# Validation
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total

    print(f"Epoch [{epoch+1}/{num_epochs}]  "
          f"Train Loss: {epoch_loss:.4f}  "
          f"Val Acc: {val_acc:.4f}")

print("Training complete!")

Epoch [1/5]  Train Loss: 2.7413  Val Acc: 0.1362
